### Wheat Image Clacification

In [1]:
#Import Libraries
import ipywidgets as widgets
from ipywidgets import *
import os
import keras
import numpy as np
from keras import layers
from keras import models
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from IPython.display import Image
from IPython import display
from tensorflow.keras.models import load_model

In [2]:
#def model architecture 
def create_model():
    model = models.Sequential()
    # first CNN layer
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    # pooling layer
    model.add(layers.MaxPooling2D((2, 2)))
    # using dropout to avoid overfitting
    model.add(layers.Dropout(0.2))
    # second  CNN layer
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.2))
    # third CNN layer
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    # flatten the output of CNN 
    model.add(layers.Flatten())
    # full connection layer
    model.add(layers.Dense(64, activation='relu'))
    # output layer
    model.add(layers.Dense(4, activation='softmax'))
    return model
# load saved weights
path='cp.ckpt'
model=create_model()
model.load_weights(path)

In [3]:
#read text file that contains the information about cure
f = open('cure.txt')
#read the file contents 
l = [ line.rstrip() for line in f.readlines()]
#group the list by sections of cure
from itertools import groupby
text=[list(group) for k, group in groupby(l, lambda x: x == "$") if not k] #k stands for the boolean
# make a dictionay contains each section and the data coresponds to it 
k=['Future guide','Leaf rust','Stem rust','Cultural practice','chemical formulations']
d = dict()
for k, v in zip(k, text):
    d[k]=v

In [4]:
def print_all(d,value):
    out_pl.clear_output()
    for k in d:
        if k == value:
            continue
        for i in d[k]:
            print(i)

In [5]:
def load_my_image(path=''):
    data=btn_upload.metadata[0]
    img_name=data['name']
    path=os.getcwd()
    path=path+'/test_images/'+img_name
    test_image = load_img(path, target_size = (64, 64))
    return test_image

In [6]:
# function to read the uploaded image and return it 
def show_my_image(path=''):
    # read the data of the file
    data = btn_upload.metadata[0]
    # get file name
    img_name = data['name']
    # get current working directory
    path = os.getcwd()
    # the complete path of the image
    path = path+'/test_images/'+img_name
    # load the image and store it 
    img = load_img(path, target_size = (400, 400))
    #return the image 
    return img

In [7]:
# function to return the prediction of the image 
def pred_img():
    # load the image 
    image = load_my_image()
    # convert image to array to feed it to the model
    image = img_to_array(image)
    # expand image dimension to the batch size for model to be able to predict
    image = np.expand_dims(image, axis = 0)
    # predict the result
    result = model.predict(image)
    # return result
    return result

In [8]:
class_indecies={0:'healthy_wheat',1:'leaf_rust',2:'stem_rust',3:'unknown'}
#k=['Future guide','Leaf rust','Stem rust','Cultural practice','chemical formulations']
# action for the button to run the model on uploaded image 
def on_click_classify(change):
    #predict the result
    pred = pred_img()
    # set the value of the label to the reuslt of  model prediction
    index = 0
    for i in range(4):
        if int(pred_img()[0][i]):
            index=i
    # show the class indeceis 
    lbl_pred.value = "Prediction : " + class_indecies[index]
    with cure :
        for k in d:
            if class_indecies[index] == 'unknown':
                break
            elif class_indecies[index] == 'healthy_wheat':
                for i in d['Future guide']:
                    print(i)
                break
            elif class_indecies[index] == 'stem_rust':
                print_all(d,'Leaf rust')
                break
            elif class_indecies[index] == 'leaf_rust':
                print_all(d,'Stem rust')
                break
            
    # clear the output
    out_pl.clear_output()
    # display the uploaded image
    with out_pl: display.display(show_my_image())

In [9]:
# Elements Of the App
# upload Button
btn_upload = widgets.FileUpload()
# output widget to display image 
out_pl = widgets.Output()
#output image to display cure 
cure = widgets.Output()
# label for the result of prediction
lbl_pred = widgets.Label()
# Button to classify the uploaded image 
btn_run = widgets.Button(description='Classify')
# event for classify button to predict the result and display uploaded image
btn_run.on_click(on_click_classify)

In [11]:
VBox([widgets.Label('Select Image'),btn_upload,btn_run,out_pl,lbl_pred,cure],box_style='info')

VBox(box_style='info', children=(Label(value='Select Image'), FileUpload(value={'healthy-wheat-crop-growing-sp…